In [1]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from nltk.tokenize.toktok import ToktokTokenizer
import spacy
import nltk
import re
from spacy.cli import download
import unicodedata

In [2]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import matutils, models, corpora, similarities
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
from wordcloud import WordCloud
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [3]:
download('en_core_web_sm')
nlp=spacy.load('en_core_web_sm')
tokenizer=ToktokTokenizer()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
picklefile=open('stack_dataframe_cleaned-1','rb')
stack_df=pickle.load(picklefile)
picklefile.close()

In [12]:
stack_df.head(3)

,level_0,index,id,id_1,title,body,owner_user_id,tags,view_count,display_name,...,views,answer_body,parent_id,owner_user_id_1,new_body,qa,text,answer,answer_unlink,list_tags
0,17050,1635517,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,...,41691,"<p>Direct from the Py3k docs: <a href=""http://...",395451,37522.0,I have a script that I'd like to continue usin...,How to download a file over http with authoriz...,"[download, file, authorization, python, workin...",Direct from the Py3k docs: http://docs.python....,direct py3k docs,"[python, python-3.x, urllib]"
3,35281,3345402,2225682,19009932,Import arbitrary python source file. (Python 3...,<p>How can I import an arbitrary python source...,2225682,python|python-3.x|python-3.3,39887,falsetru,...,18449,<p><strong><code>importlib</code> helper funct...,19009932,895245.0,How can I import an arbitrary python source fi...,Import arbitrary python source file. (Python 3...,"[import, arbitrary, python, source, file, pyth...","helper function\nHere is a convenient, ready-...",helper function convenient ready use helper re...,"[python, python-3.x, python-3.3]"
7,52323,4936459,298479,9039333,How to run an inline shell script from python?,<p>I have a small shell script in a string ins...,298479,python|bash,2476,ThiefMaster,...,39117,"<pre><code>import subprocess\n\nscript = """"""\n...",9039333,17160.0,I have a small shell script in a string inside...,How to run an inline shell script from python?...,"[run, inline, shell, script, python, small, sh...",,,"[python, bash]"


In [ ]:
#n_topics=20'#min_df originally 10
tf_vect=CountVectorizer(max_df=0.8, min_df=5, stop_words='english',token_pattern=r'(?u)\b[A-Za-z]+\b')
#max_features=1000, 
tf=tf_vect.fit_transform(stack_df['qa']).toarray()
tf_vect.get_feature_names()

In [ ]:
X=stack_df['text']
y=stack_df['list_tags'].apply(lambda x: list(x))

In [5]:
vect=TfidfVectorizer(analyzer='word', max_df=.9,min_df=0.01,tokenizer=None,
                    preprocessor=' '.join,stop_words=None,lowercase=False)

In [10]:
vect.fit(X)
X_tfidf=vect.fit_transform(X).toarray()
#print(X_tfidf)

In [ ]:
len(vect.get_feature_names())

In [ ]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    #return pd.DataFrame.from_dict(topic_dict,orient='index')
    return pd.DataFrame(topic_dict)

In [ ]:
topics=display_topics(lda, tf_vect.get_feature_names_out(), no_top_words)

In [ ]:
topics

In [ ]:
plt.scatter(topics['Topic 0 weights'],y=topics['Topic 0 words'])
plt.scatter(topics['Topic 1 weights'],y=topics['Topic 1 words'])

In [ ]:

picklefile=open('tsne_2dim_count','rb')
tsne_features1=pickle.load(picklefile)
picklefile.close()

In [ ]:
def get_data(k,num_points=100):
    np.random.seed(9)
    data = []
    for i in range(0,k):
        for _ in range(0,num_points):
            data.append([tsne_features[:,0],tsne_features[:,1]])
    x1,y1 = zip(*data)
    #plt.xlabel('X')
    #plt.ylabel('Y')
    #plt.scatter(x1,y1)
    return data

In [ ]:
data = get_data(6)
x,y = zip(*data)
plt.scatter(x, y, s=20);

In [ ]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets import make_blobs

plt.figure(dpi=200)
centers = [[1, 1], [-1, -1], [1, -1]]
#X, _ = make_blobs(n_samples=10000, centers=centers, cluster_std=0.6)
x,y = zip(*data)
plt.scatter(x,y);

In [ ]:
# estimate bandwidth 
# as mentioned in module, bigger window would allow for smoother groupings, 
# We can think of the bandwidth as our kernel's radius, 
bandwidth = estimate_bandwidth(X, n_samples=500)
print('bandwidth: %d' % bandwidth)

#ms = MeanShift(bandwidth=bandwidth,bin_seeding=True)
ms = MeanShift(bin_seeding=True)   # in the case where we do not define any parameters
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

In [ ]:
#tsne_features5=model.fit_transform(stack_df[['reputation','view_count']])

In [ ]:
#from office hours nlp_pipline-ipynb
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)
#try to play around with less topics top 5 words, top 6 words
no_top_words = 30
topic_df=display_topics(lda, vect.get_feature_names_out(), no_top_words)

In [ ]:
#https://towardsdatascience.com/explaining-dbscan-clustering-18eaf5c83b31

In [ ]:
#tf-count vect-tfidf
X_scale = StandardScaler().fit(tf).transform(tf)

In [ ]:
df_scale_count=pd.DataFrame(X_scale,columns=tf_vect.get_feature_names())

In [ ]:
df_scale_count.head(3)

In [ ]:
#https://towardsdatascience.com/clustering-documents-with-python-97314ad6a78d
pca=PCA(n_components=2,random_state=42)
pca.fit(df_scale_count)
variance=pca.explained_variance_ratio_
reduced_features=pca.fit_transform(X_tfidf)
var=np.cumsum(np.round(variance,3)*100)

In [ ]:
plt.figure(figsize=(12,5))
plt.ylabel('% Variance Explained')
plt.xlabel('# of features')
plt.title('PCA Analysis')
plt.ylim(0,20)
plt.plot(var)

In [ ]:
from sklearn.cluster import KMeans
sum_of_squared_distances=[]
K=range(2,50)
for k in K:
    km=KMeans(n_clusters=k,max_iter=200,n_init=10)
    km=km.fit(X_tfidf)
    sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K,sum_of_squared_distances,'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method for Optimal K')
plt.show()

In [ ]:
true_k=30
model=KMeans(n_clusters=true_k,init='k-means++',max_iter=200,n_init=10)
model.fit(X_tfidf)
labels=model.labels_
topics_cl=pd.DataFrame(list(zip(stack_df['qa'],labels)),columns=['topic','cluster'])

In [ ]:
print(topics_cl.sort_values(by=['cluster']))
topic_sort=topics_cl.sort_values(by=['cluster'])
topic_sort


In [ ]:
topic_sort.index

In [ ]:
len(model.labels_)

In [ ]:
#reduced_features

In [ ]:
reduced_cluster_center=pca.transform(model.cluster_centers_)

In [ ]:
from scipy.stats import gaussian_kde
#z=gaussian_kde(reduced_features)

In [ ]:
reduced_cluster_center

In [ ]:
#plt.scatter(reduced_features[:,0], reduced_features[:,1], c=model.predict(X_tfidf))
plt.scatter(reduced_cluster_center[:, 0], reduced_cluster_center[:,1], marker='x', s=150, c='b')
plt.xlabel('PCA_component 1 Python')
plt.ylabel('PCA_component 2 Miscellaneous Python')
plt.title('Cluster Centers from reduced dimension of the TFIDF Matrix ')
plt.savefig(f'Cluster centers from reduced dimension of tfidf matrix.jpg',bbox_inches='tight',dpi=300)

In [ ]:
plt.scatter(reduced_features[:,0], reduced_features[:,1], c=model.predict(X_tfidf),alpha=0.2)
plt.xlabel('PCA_component 1 Python')
plt.ylabel('PCA_component 2 Miscellaneous Python')
plt.title('Scatter plot of reduced dimension of the TFIDF Matrix ')
plt.savefig(f'Scatter plot of reduced dimension of the TFIDF Matrix.jpg',bbox_inches='tight',dpi=300)

In [ ]:
from sklearn.metrics import homogeneity_score
homogeneity_score(labels,model.predict(X_tfidf))

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X_tfidf,labels=model.predict(X_tfidf))

In [ ]:
from wordcloud import WordCloud
result={'cluster':labels,'topic':stack_df['qa']}
result=pd.DataFrame(result)
for k in range(0,true_k):
   s=result[result.cluster==k]
   text=s['topic'].str.cat(sep=' ')
   text=text.lower()
   text=' '.join([word for word in text.split()])
   wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
   print('Cluster: {}'.format(k))
   print('Titles')
   titles=topics_cl[topics_cl.cluster==k]['topic']         
   #print(titles.to_string(index=False))
   plt.figure()
   plt.imshow(wordcloud, interpolation="bilinear")
   plt.axis("off")
   plt.show()